In [31]:
import torch
import torch.nn as nn

In [32]:
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden_layer1 = nn.Linear(2, 2)  # First hidden layer
        self.hidden_layer2 = nn.Linear(2, 2)  # Second hidden layer
        self.output_layer = nn.Linear(2, 2)   # Output layer
        self._initialize_weights()

    def _initialize_weights(self):
        with torch.no_grad():
            # Weights and biases based on the image
            self.hidden_layer1.weight.copy_(torch.tensor([[0.15, 0.20], [0.25, 0.30]]))
            self.hidden_layer1.bias.copy_(torch.tensor([0.35, 0.35]))
            self.hidden_layer2.weight.copy_(torch.tensor([[0.40, 0.45], [0.50, 0.55]]))
            self.hidden_layer2.bias.copy_(torch.tensor([0.60, 0.60]))
            self.output_layer.weight.copy_(torch.tensor([[0.593269992, 0.75136507], [0.596884378, 0.772928465]]))
            self.output_layer.bias.copy_(torch.tensor([0.35, 0.60]))

    def forward(self, x):
        # Forward pass
        self.hidden_input1 = self.hidden_layer1(x)
        self.hidden_output1 = torch.sigmoid(self.hidden_input1)

        self.hidden_input2 = self.hidden_layer2(self.hidden_output1)
        self.hidden_output2 = torch.sigmoid(self.hidden_input2)

        self.final_input = self.output_layer(self.hidden_output2)
        self.final_output = torch.sigmoid(self.final_input)
        return self.final_output

In [33]:
model = CustomModel()
input_tensor = torch.tensor([0.05, 0.10])  # Input values from the image
target = torch.tensor([0.99, 0.01])        # Target values from the image
output = model(input_tensor)

In [34]:
loss = 0.5 * torch.sum((target - output) ** 2)
print("Loss:", loss.item())

learning_rate = 0.4
delta_output = (output - target) * (output * (1 - output))

Loss: 0.3614194691181183


In [35]:
with torch.no_grad():
    for i in range(2):
        for j in range(2):
            model.output_layer.weight[i][j] -= learning_rate * delta_output[i] * model.hidden_output2[j]
        model.output_layer.bias[i] -= learning_rate * delta_output[i]

In [36]:
delta_hidden2 = torch.zeros(2)
for i in range(2):
    delta_hidden2[i] = (delta_output[0] * model.output_layer.weight[0][i] +
                        delta_output[1] * model.output_layer.weight[1][i]) * (model.hidden_output2[i] * (1 - model.hidden_output2[i]))

In [37]:
with torch.no_grad():
    for i in range(2):
        for j in range(2):
            model.hidden_layer2.weight[i][j] -= learning_rate * delta_hidden2[i] * model.hidden_output1[j]
        model.hidden_layer2.bias[i] -= learning_rate * delta_hidden2[i]

In [38]:
delta_hidden1 = torch.zeros(2)
for i in range(2):
    delta_hidden1[i] = (delta_hidden2[0] * model.hidden_layer2.weight[0][i] +
                        delta_hidden2[1] * model.hidden_layer2.weight[1][i]) * (model.hidden_output1[i] * (1 - model.hidden_output1[i]))

In [39]:
with torch.no_grad():
    for i in range(2):
        for j in range(2):
            model.hidden_layer1.weight[i][j] -= learning_rate * delta_hidden1[i] * input_tensor[j]
        model.hidden_layer1.bias[i] -= learning_rate * delta_hidden1[i]


In [40]:
print("Hidden Layer 1 Weights:", model.hidden_layer1.weight)
print("Hidden Layer 1 Biases:", model.hidden_layer1.bias)
print("Hidden Layer 2 Weights:", model.hidden_layer2.weight)
print("Hidden Layer 2 Biases:", model.hidden_layer2.bias)
print("Output Layer Weights:", model.output_layer.weight)
print("Output Layer Biases:", model.output_layer.bias)

Hidden Layer 1 Weights: Parameter containing:
tensor([[0.1500, 0.1999],
        [0.2500, 0.2999]], requires_grad=True)
Hidden Layer 1 Biases: Parameter containing:
tensor([0.3492, 0.3491], requires_grad=True)
Hidden Layer 2 Weights: Parameter containing:
tensor([[0.3980, 0.4480],
        [0.4975, 0.5475]], requires_grad=True)
Hidden Layer 2 Biases: Parameter containing:
tensor([0.5967, 0.5958], requires_grad=True)
Output Layer Weights: Parameter containing:
tensor([[0.6025, 0.7609],
        [0.5631, 0.7382]], requires_grad=True)
Output Layer Biases: Parameter containing:
tensor([0.3623, 0.5551], requires_grad=True)
